# Machine Learning via Generalized Linear Model (GLM) and Firth Logistic Regression (FLR)

The purpose is to generate a new data-table, whose row is the unique characteristic of each donor, containing the ID, the type (AD or control), the subclass and the expression condition of each gene etc.
To do this, the first thing to do is to arrange the dispersive single-cell dataset into the form of the targeted new data-table.

Import the packages we want:

In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import os

Load the original data:

In [2]:
metadata = pd.read_excel("/Users/a1234/Desktop/Donor metadata.xlsx")
mtg = pd.read_excel("/Users/a1234/Desktop/Quantitative neuropathology summary data (MTG).xlsx")
omics_folder = "/Users/a1234/Desktop/Omics data (scRNA-seq)/"

Format example:

In [3]:
omics_example = sc.read_h5ad("/Users/a1234/Desktop/Omics data (scRNA-seq)/H21.33.002_SEAAD_MTG_RNAseq_final-nuclei.2024-02-13.h5ad")

In [4]:
omics_example

AnnData object with n_obs × n_vars = 15761 × 36601
    obs: 'sample_id', 'Neurotypical reference', 'Donor ID', 'Organism', 'Brain Region', 'Sex', 'Gender', 'Age at Death', 'Race (choice=White)', 'Race (choice=Black/ African American)', 'Race (choice=Asian)', 'Race (choice=American Indian/ Alaska Native)', 'Race (choice=Native Hawaiian or Pacific Islander)', 'Race (choice=Unknown or unreported)', 'Race (choice=Other)', 'specify other race', 'Hispanic/Latino', 'Highest level of education', 'Years of education', 'PMI', 'Fresh Brain Weight', 'Brain pH', 'Overall AD neuropathological Change', 'Thal', 'Braak', 'CERAD score', 'Overall CAA Score', 'Highest Lewy Body Disease', 'Total Microinfarcts (not observed grossly)', 'Total microinfarcts in screening sections', 'Atherosclerosis', 'Arteriolosclerosis', 'LATE', 'Cognitive Status', 'Last CASI Score', 'Interval from last CASI in months', 'Last MMSE Score', 'Interval from last MMSE in months', 'Last MOCA Score', 'Interval from last MOCA in mont

We now generate the targeted dataset which is used for further machine learning:

Firstly, we deal with the omics data:

In [5]:
omics_files = [f for f in os.listdir(omics_folder) if f.endswith('.h5ad')]

donor_ids = [f.split('_')[0] for f in omics_files]

omics_data = []

for i, file in enumerate(omics_files):
    adata = sc.read_h5ad(os.path.join(omics_folder, file))
    
    obs_data = adata.obs[["Thal", "Braak", "Overall CAA Score", "Last MMSE Score", "NeuN positive fraction on FANS", "Fraction mitochondrial UMIs", "Number of UMIs", "Doublet score", "Class", "Subclass"]].copy()
    
    obs_data['Donor_ID'] = donor_ids[i]
    
    omics_data.append(obs_data)

omics_data_df = pd.concat(omics_data)

# omics_data_df.to_csv("omics_data(with gene related).csv", index=False)

In [6]:
omics_data_df = omics_data_df.rename(columns={'Donor_ID': 'Donor ID'})

In [7]:
# Extract Donor_ID from Omics data
donor_ids = [file.split('_')[0] for file in omics_files if file.endswith('.h5ad')]

In [8]:
# Find out the common Donor_ID
common_donor_ids = set(metadata['Donor ID']).intersection(set(mtg['Donor ID'])).intersection(set(donor_ids))

In [9]:
# For metadata
metadata_filtered = metadata[metadata['Donor ID'].isin(common_donor_ids)][['Donor ID', 'Age at Death', 'Sex', 'Highest level of education', 'Cognitive Status', 'Overall AD neuropathological Change','PMI', 'RIN','LATE', 'Years of education']]

In [13]:
# Deal with the corresponding 'Race' columns in metadata
race_columns = [col for col in metadata.columns if 'Race' in col]

race_info = metadata[['Donor ID'] + race_columns].copy()

race_info.loc[:,'Race'] = race_info.apply(get_race, axis=1)

metadata_filtered = pd.merge(metadata_filtered, race_info[['Donor ID', 'Race']], on='Donor ID', how='left')

In [14]:
# Deal with the corresponding 'Consensus Clinical Dx' columns in metadata
ccdx_columns = [col for col in metadata.columns if 'Consensus Clinical Dx' in col]

ccdx_info = metadata[['Donor ID'] + ccdx_columns]

ccdx_info = metadata[['Donor ID'] + ccdx_columns].copy()

ccdx_info.loc[:,'Consensus Clinical Dx'] = ccdx_info.apply(get_ccdx, axis=1)

metadata_filtered = pd.merge(metadata_filtered, ccdx_info[['Donor ID', 'Consensus Clinical Dx']], on='Donor ID', how='left')

Now, we deal with the quantitative neuropathology summary data (MTG):

In this part, we select some of the columns of MTG based on domain knowledge.

In [15]:
# Extract useful data from Quantitative neuropathology summary data
mtg_filtered = mtg.loc[
    mtg['Donor ID'].isin(common_donor_ids),  
    ['Donor ID',
    'total AT8 positive area_Grey matter', 
    'number of AT8 positive cells_Grey matter', 
    'average AT8 positive cell area_Grey matter',
    'total pTDP43 positive area_Grey matter', 
    'number of pTDP43 positive cells_Grey matter',
    'total Iba1 positive area_Grey matter', 
    'number of Iba1 positive cells_Grey matter', 
    'number of activated Iba1 positive cells_Grey matter',
    'number of 6e10 positive objects_Grey matter'
    ]
]

In [16]:
merged_data = pd.merge(metadata_filtered, mtg_filtered, on = 'Donor ID', how = 'inner')
merged_data = pd.merge(merged_data, omics_data_df, on = 'Donor ID', how = 'inner')

In [20]:
merge_data = merged_data.copy()

continuous_cols = merge_data.select_dtypes(include=[np.number]).columns.tolist()

categorical_cols = merge_data.select_dtypes(exclude=[np.number]).columns.tolist()
categorical_cols = [c for c in categorical_cols if c != "Donor ID"]

cont_agg = merge_data.groupby("Donor ID")[continuous_cols].mean()

def mode_or_unknown(x):
    m = x.mode()
    return m.iloc[0] if not m.empty else "Unknown"

cat_agg = merge_data.groupby("Donor ID")[categorical_cols].agg(mode_or_unknown)

subclass_counts = (
    merge_data.groupby(["Donor ID", "Subclass"],observed=False)
    .size()
    .unstack(fill_value=0)
)
subclass_freq = subclass_counts.div(subclass_counts.sum(axis=1), axis=0)

class_counts = (
    merge_data.groupby(["Donor ID", "Class"],observed=False)
    .size()
    .unstack(fill_value=0)
)
class_freq = class_counts.div(subclass_counts.sum(axis=1), axis=0)

donor_level_df = cont_agg.join(cat_agg).join(subclass_freq).join(class_freq)

“donor_level_df” is the washed dataset and it is the original dataset we will use in the machine learning:
Note that we have already transform the cell-level data into donor-level data, and by doing this, we do the above steps:
For the continuous variables: we use the mean value to replace the individual values;
For separated variable: we count the frequencies of each of them (here, they are the columns --- "Class" and "Subclass");
For others: we select the mode.


Functions used in the process of data preparation:

In [21]:
# Deal with the information of the set of 'Race' in metadata
def get_race(row):
    races = []
    for col in race_columns:
        if row[col] == 'Checked':
            race = col.split('choice=')[1].split(')')[0]
            races.append(race)
    return ', '.join(races) if races else 'Unknown'

# Deal with the information of the set of 'Consensus Clinical Dx' in metadata
def get_ccdx(row):
    ccdx = []
    for col in ccdx_columns:
        if row[col] == 'Checked':
            diagnosis = col.split('choice=')[1].split(')')[0]
            ccdx.append(diagnosis)
    return ', '.join(ccdx) if ccdx else 'Unknown'

## Machine Learning through pipeline

In [51]:
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, make_scorer, roc_auc_score, average_precision_score, brier_score_loss, make_scorer
from sklearn.linear_model import LogisticRegression  
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GroupKFold, GridSearchCV

In [52]:
# Defining features: Donor-level data, excluding the target variable "Cognitive Status" and "Donor ID"
X_raw = donor_level_df.drop('Cognitive Status', axis=1).reset_index()
Y = donor_level_df['Cognitive Status'].values
groups = X_raw['Donor ID'].values
X = X_raw.drop('Donor ID', axis=1)

In [69]:
X

,Age at Death,PMI,RIN,Years of education,total AT8 positive area_Grey matter,number of AT8 positive cells_Grey matter,average AT8 positive cell area_Grey matter,total pTDP43 positive area_Grey matter,number of pTDP43 positive cells_Grey matter,total Iba1 positive area_Grey matter,...,Pax6,Pvalb,Sncg,Sst,Sst Chodl,VLMC,Vip,Neuronal: GABAergic,Neuronal: Glutamatergic,Non-neuronal and Non-neural
0,80.0,8.133333,8.33,17.0,1.047120e+04,0.0,0.000000,0.000000,0.0,3552174.0,...,0.008604,0.079063,0.012164,0.051660,0.002108,0.003352,0.086561,0.295622,0.521373,0.183006
1,82.0,7.700000,8.60,16.0,5.798206e+04,42.0,102.924044,2876.068832,24.0,4237830.0,...,0.004955,0.062660,0.009359,0.018818,0.000801,0.004604,0.071168,0.212402,0.615735,0.171863
2,97.0,4.333333,7.87,12.0,7.280875e+03,1.0,26.316354,0.000000,0.0,1790954.0,...,0.008414,0.064703,0.022922,0.031530,0.001016,0.001499,0.069491,0.253397,0.584748,0.161855
3,86.0,8.833333,7.83,15.0,7.556641e+05,371.0,80.896970,0.000000,0.0,4284872.0,...,0.006153,0.072510,0.015524,0.027736,0.001136,0.003171,0.074404,0.251751,0.595418,0.152830
4,99.0,7.600000,8.40,12.0,8.481321e+04,1.0,35.419410,0.000000,0.0,2984593.0,...,0.007098,0.063704,0.015899,0.020571,0.000652,0.004455,0.089816,0.258293,0.584673,0.157033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,95.0,4.400000,7.27,16.0,2.739650e+04,6.0,77.544874,0.000000,0.0,2527673.0,...,0.008264,0.073509,0.024392,0.041586,0.001000,0.002932,0.107298,0.316295,0.481106,0.202599
79,88.0,7.000000,8.63,15.0,1.009349e+06,199.0,82.742644,0.000000,0.0,6367609.0,...,0.012921,0.042431,0.026320,0.028075,0.000319,0.005264,0.083107,0.233371,0.525443,0.241187
80,94.0,4.000000,6.55,12.0,6.336955e+06,1561.0,69.949808,32320.361940,960.0,5981221.0,...,0.013160,0.045036,0.039187,0.027782,0.000292,0.016815,0.087586,0.305746,0.471122,0.223132
81,97.0,7.000000,7.28,17.0,2.329598e+05,88.0,83.979676,0.000000,0.0,977078.0,...,0.006675,0.072370,0.024943,0.027578,0.001230,0.005621,0.115054,0.324785,0.499210,0.176006


In [70]:
Y

array(['No dementia', 'No dementia', 'No dementia', 'Dementia',
       'No dementia', 'No dementia', 'Dementia', 'No dementia',
       'No dementia', 'No dementia', 'Dementia', 'Dementia', 'Dementia',
       'Dementia', 'No dementia', 'Dementia', 'No dementia',
       'No dementia', 'No dementia', 'Dementia', 'Dementia',
       'No dementia', 'Dementia', 'No dementia', 'Dementia',
       'No dementia', 'No dementia', 'No dementia', 'Dementia',
       'Dementia', 'No dementia', 'Dementia', 'Dementia', 'No dementia',
       'No dementia', 'Dementia', 'Dementia', 'Dementia', 'Dementia',
       'No dementia', 'No dementia', 'Dementia', 'No dementia',
       'Dementia', 'Dementia', 'Dementia', 'Dementia', 'No dementia',
       'Dementia', 'Dementia', 'No dementia', 'No dementia', 'Dementia',
       'Dementia', 'Dementia', 'No dementia', 'Dementia', 'Dementia',
       'No dementia', 'No dementia', 'No dementia', 'No dementia',
       'Dementia', 'No dementia', 'Dementia', 'No dementia', 'Dem

In [53]:
# Define the cross-validation strategy (5 fold)
N_SPLITS = 5
outer_cv = GroupKFold(n_splits=N_SPLITS)
inner_cv = GroupKFold(n_splits=N_SPLITS)

In [55]:
classifiers = {
    'L2 Regression (CV)': GridSearchCV(
        estimator=LogisticRegression(
            penalty='l2', solver='lbfgs', class_weight='balanced',
            max_iter=3000, random_state=42
        ),
        param_grid={'C': np.logspace(-4, 1, 10)},
        cv=inner_cv, scoring='roc_auc'
    ),
    'ElasticNet Regression (CV)': GridSearchCV(
        estimator=LogisticRegression(
            penalty='elasticnet', solver='saga', class_weight='balanced',
            max_iter=3000, random_state=42
        ),
        param_grid={
            'C': np.logspace(-4, 1, 10),
            'l1_ratio': [0.25, 0.5, 0.75]
        },
        cv=inner_cv, scoring='roc_auc'
    ),
    # Aiming to mimicking the Firth Logistic Regression (by setting C = 0.01)
    'L2 Regression (Strong Shrinkage C=0.01)': LogisticRegression( 
        penalty='l2', C=0.01, solver='lbfgs', class_weight='balanced',
        max_iter=3000, random_state=42
    )
}

In [56]:
# Define the feature preprocessing pipeline
full_feature_list = FEATURE_MAP['M'] + FEATURE_MAP['Q'] + FEATURE_MAP['O']
X_full = X[full_feature_list]

numerical_features_full = X_full.select_dtypes(include=['number']).columns.tolist()
categorical_features_full = X_full.select_dtypes(exclude=['number']).columns.tolist()

full_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features_full),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features_full)
    ],
    remainder='passthrough'
)

In [58]:
# 包含插补的完整数值和分类转换器
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
full_preprocessor_with_impute = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features_full),
        ('cat', categorical_transformer, categorical_features_full)
    ]
)

In [64]:
# Evaluate the performance of different models
model_results = {}

for name, model in classifiers.items():
    print(f"\n--- Evaluating Model: {name} ---")
    
    pipeline = Pipeline(steps=[
        ('preprocessor', full_preprocessor_with_impute),
        ('classifier', model)
    ])
    
    accuracies, precisions, recalls, f1s = [], [], [], []
    aucs, pr_aucs, brier_scores = [], [], []

    for fold, (train_idx, test_idx) in enumerate(outer_cv.split(X_full, Y, groups)):
        X_train, X_test = X_full.iloc[train_idx], X_full.iloc[test_idx]
        y_train, y_test = Y[train_idx], Y[test_idx]
        groups_train = groups[train_idx]
        
        if isinstance(model, GridSearchCV):
            pipeline.fit(X_train, y_train, classifier__groups=groups_train)
        else:
            pipeline.fit(X_train, y_train)

        pos_label = 'Dementia'
        pos_index = list(pipeline.classes_).index(pos_label)
        y_pred_proba = pipeline.predict_proba(X_test)[:, pos_index]
        y_pred = pipeline.predict(X_test) 
        
        y_test_bin = np.where(y_test == pos_label, 1, 0)
        
        accuracies.append(accuracy_score(y_test, y_pred)) 
        precisions.append(precision_score(y_test, y_pred, pos_label=pos_label, zero_division=0)) 
        recalls.append(recall_score(y_test, y_pred, pos_label=pos_label, zero_division=0)) 
        f1s.append(f1_score(y_test, y_pred, pos_label=pos_label, zero_division=0)) 
        
        aucs.append(roc_auc_score(y_test_bin, y_pred_proba))
        pr_aucs.append(average_precision_score(y_test_bin, y_pred_proba))
        brier_scores.append(brier_score_loss(y_test_bin, y_pred_proba))

    auc_mean = np.mean(aucs)
    auc_std = np.std(aucs)

    auc_se = auc_std / np.sqrt(len(aucs))
    
    auc_ci_low = auc_mean - 1.96 * auc_se
    auc_ci_high = auc_mean + 1.96 * auc_se
    
    model_results[name] = {
        'Accuracy': f"{np.mean(accuracies):.3f} ± {np.std(accuracies):.3f}", 
        'Precision': f"{np.mean(precisions):.3f} ± {np.std(precisions):.3f}", 
        'Recall': f"{np.mean(recalls):.3f} ± {np.std(recalls):.3f}", 
        'F1-score': f"{np.mean(f1s):.3f} ± {np.std(f1s):.3f}",
        'AUC': auc_mean, 
        'AUC (95% CI)': f"({auc_ci_low:.3f} - {auc_ci_high:.3f})", 
        'PR-AUC': f"{np.mean(pr_aucs):.3f} ± {np.std(pr_aucs):.3f}", 
        'Brier Score': f"{np.mean(brier_scores):.3f} ± {np.std(brier_scores):.3f}" 
    }

print("\n--- Model Comparison Results ---")
model_results_df = pd.DataFrame.from_dict(model_results, orient='index')

display_cols = ['Accuracy', 'Precision', 'Recall', 'F1-score', 'AUC', 'AUC (95% CI)', 'PR-AUC', 'Brier Score']
print(model_results_df[display_cols].sort_values(by='AUC', ascending=False))


--- Evaluating Model: L2 Regression (CV) ---

--- Evaluating Model: ElasticNet Regression (CV) ---

--- Evaluating Model: L2 Regression (Strong Shrinkage C=0.01) ---

--- Model Comparison Results ---
                                              Accuracy      Precision  \
L2 Regression (CV)                       0.829 ± 0.075  0.868 ± 0.113   
ElasticNet Regression (CV)               0.830 ± 0.049  0.847 ± 0.088   
L2 Regression (Strong Shrinkage C=0.01)  0.771 ± 0.061  0.836 ± 0.083   

                                                Recall       F1-score  \
L2 Regression (CV)                       0.803 ± 0.102  0.826 ± 0.072   
ElasticNet Regression (CV)               0.825 ± 0.127  0.825 ± 0.058   
L2 Regression (Strong Shrinkage C=0.01)  0.678 ± 0.173  0.733 ± 0.099   

                                              AUC     AUC (95% CI)  \
L2 Regression (CV)                       0.918849  (0.850 - 0.987)   
ElasticNet Regression (CV)               0.918849  (0.859 - 0.979)   
L2 

In [65]:
# Evaluate the incremental value of different feature combinations
# Here we select the best-performing model --- ElasticNet Regression (CV)
best_model = classifiers['ElasticNet Regression (CV)']

block_combinations = {
    'M-Only (Clinical Baseline)': FEATURE_MAP['M'],
    'M+Q': FEATURE_MAP['M'] + FEATURE_MAP['Q'],
    'M+O': FEATURE_MAP['M'] + FEATURE_MAP['O'],
    'M+Q+O (Full Model)': full_feature_list
}

In [68]:
results_by_block = {}

for name, feature_list in block_combinations.items():

    X_block = X[feature_list]
    
    numerical_features_block = X_block.select_dtypes(include=['number']).columns.tolist()
    categorical_features_block = X_block.select_dtypes(exclude=['number']).columns.tolist()

    block_preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_features_block),
            ('cat', categorical_transformer, categorical_features_block)
        ],
        remainder='passthrough'
    )
    
    pipeline = Pipeline(steps=[
        ('preprocessor', block_preprocessor),
        ('classifier', best_model) 
    ])

    aucs, pr_aucs, brier_scores = [], [], []

    for fold, (train_idx, test_idx) in enumerate(outer_cv.split(X_block, Y, groups)):
        X_train, X_test = X_block.iloc[train_idx], X_block.iloc[test_idx]
        y_train, y_test = Y[train_idx], Y[test_idx]
        groups_train = groups[train_idx]
        
        pipeline.fit(X_train, y_train, classifier__groups=groups_train)
        
        pos_label = 'Dementia'
        pos_index = list(pipeline.classes_).index(pos_label)
        y_pred_proba = pipeline.predict_proba(X_test)[:, pos_index]
        y_test_bin = np.where(y_test == pos_label, 1, 0)
        
        aucs.append(roc_auc_score(y_test_bin, y_pred_proba))
        pr_aucs.append(average_precision_score(y_test_bin, y_pred_proba))
        brier_scores.append(brier_score_loss(y_test_bin, y_pred_proba))
        
    results_by_block[name] = {
        'AUC Mean': np.mean(aucs),
        'AUC Std': np.std(aucs),
        'PR-AUC Mean': np.mean(pr_aucs),
        'Brier Score Mean': np.mean(brier_scores),
    }

In [67]:
print("\n--- Feature Block Incremental Value Results ---")
results_df = pd.DataFrame.from_dict(results_by_block, orient='index')
results_df['Features'] = [len(block_combinations[name]) for name in results_df.index]
print(results_df[['AUC Mean', 'AUC Std', 'Brier Score Mean', 'Features']].sort_values(by='AUC Mean', ascending=False))


--- Feature Block Incremental Value Results ---
                            AUC Mean   AUC Std  Brier Score Mean  Features
M+Q                         0.937302  0.069207          0.106536        46
M-Only (Clinical Baseline)  0.926190  0.062371          0.094102        37
M+O                         0.924802  0.062025          0.095088        47
M+Q+O (Full Model)          0.918849  0.068785          0.114519        56
